In [7]:
from litellm import completion

In [16]:
MODEL_NAME = 'hosted_vllm/Qwen3-235B'
api_base  = 'http://192.168.0.166:8000/v1'

In [17]:
stream = completion(
    model=MODEL_NAME, 
    messages=[{"role": "user", "content": "法国的首都是哪"}],
    max_retries= 3,
    api_base=api_base,
    stream=True,
)

reasoning_content = ''
answer_content = ''
is_answering = False  # 是否进入回复阶段

print("\n" + "=" * 20 + "思考过程" + "=" * 20 + "\n")
if stream:
    response = ""
    for chunk in stream:
        delta = chunk.choices[0].delta
        # 收集思考内容
        if hasattr(delta, "reasoning_content") and delta.reasoning_content is not None:
                if not is_answering:
                    print(delta.reasoning_content, end="", flush=True)
                reasoning_content += delta.reasoning_content
        # 收到content，开始进行回复
        if hasattr(delta, "content") and delta.content:
            if not is_answering:
                print("\n" + "=" * 20 + "回复部分" + "=" * 20 + "\n")
                is_answering = True
            print(delta.content, end="", flush=True)
            answer_content += delta.content      
            


====================思考过程====================


好的

，用户问的是“法国的首都是哪”。首先，我需要确认用户的问题是否完整，因为问题末尾似乎缺少一个字，可能是“哪里”。不过根据常识，法国的首都众所周知是巴黎，所以用户可能只是输入时有误或者没有完成。

接下来，我需要验证这一点。法国的官方首都确实是巴黎，而巴黎也是欧洲的重要城市之一，有着丰富的历史和文化。不过有时候可能会有混淆，比如有人可能认为法国的首都不是巴黎，或者有其他城市曾经担任过首都。例如，法国历史上曾多次迁都，但自1870年以来，巴黎一直是法国的首都。

另外，要确保用户没有混淆其他同名国家，比如海地的首都也叫太子港，但法国的首都显然不同。此外，可能用户想问的是其他法语国家的首都，但根据问题中的“法国”，答案应该是巴黎。

还需要考虑用户可能的拼写错误或其他语言的问题，但中文里“法国”对应的首都就是巴黎。因此，回答应该是明确的，同时也可以补充一些关于巴黎的信息，以提供更多价值。

最后，确保回答简洁准确，同时涵盖用户可能需要的额外信息，比如历史背景或重要性，但根据用户的问题，可能只需要直接回答即可。

====================回复部分====================



法国的首都是**巴黎**。  
巴黎不仅是法国的政治、经济和文化中心，也是世界著名的城市之一，以其历史遗产、艺术氛围和地标建筑（如埃菲尔铁塔、卢浮宫等）闻名。自古以来，巴黎一直是法国的首都，从未改变。

In [26]:
from openai import OpenAI
client = OpenAI(base_url=api_base,api_key='123')

stream = client.chat.completions.create(
    model='',
    messages=[{"role": "user", "content": "法国的首都是哪"}],
    stream=True,
    temperature=0,
    extra_body={"chat_template_kwargs": {"enable_thinking": False},}
)

reasoning_content = ''
answer_content = ''
is_answering = False  # 是否进入回复阶段

print("\n" + "=" * 20 + "思考过程" + "=" * 20 + "\n")
if stream:
    response = ""
    for chunk in stream:
        delta = chunk.choices[0].delta
        # 收集思考内容
        if hasattr(delta, "reasoning_content") and delta.reasoning_content is not None:
                if not is_answering:
                    print(delta.reasoning_content, end="", flush=True)
                reasoning_content += delta.reasoning_content
        # 收到content，开始进行回复
        if hasattr(delta, "content") and delta.content:
            if not is_answering:
                print("\n" + "=" * 20 + "回复部分" + "=" * 20 + "\n")
                is_answering = True
            print(delta.content, end="", flush=True)
            answer_content += delta.content      


====================思考过程====================

法国的首都是**巴黎**。

In [29]:
from openai import OpenAI
# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"

client = OpenAI(
    api_key=openai_api_key,
    base_url=api_base,
)

chat_response = client.chat.completions.create(
    model="",
    messages=[
        {"role": "user", "content": "法国的首都"},
    ],
    max_tokens=8192,
    temperature=0.7,
    top_p=0.8,
    presence_penalty=1.5,
    extra_body={
        "top_k": 20, 
        "chat_template_kwargs": {"enable_thinking": False},
    },
)
print("Chat response:", chat_response)

Chat response: ChatCompletion(id='chatcmpl-eb1108b2d3ae4e5f883c1d97e43ff7ed', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content='法国的首都是**巴黎**。  \n\n巴黎不仅是法国的政治、经济和文化中心，也是世界著名的城市之一，以其历史遗产、艺术氛围和地标建筑（如埃菲尔铁塔、卢浮宫和巴黎圣母院）闻名。'), stop_reason=None)], created=1748250577, model='Qwen3-235B', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=53, prompt_tokens=15, total_tokens=68, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None)


In [13]:
# SPDX-License-Identifier: Apache-2.0
"""
An example shows how to generate structured outputs from reasoning models
like DeepSeekR1. The thinking process will not be guided by the JSON
schema provided by the user. Only the final output will be structured.

To run this example, you need to start the vLLM server with the reasoning
parser:

```bash
vllm serve deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B \
    --reasoning-parser deepseek_r1
```

This example demonstrates how to generate chat completions from reasoning models
using the OpenAI Python client library.
"""

from enum import Enum

from openai import OpenAI
from pydantic import BaseModel

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://192.168.0.166:8000/v1"


def print_completion_details(completion):
    print("reasoning_content: ",
          completion.choices[0].message.reasoning_content)
    print("content: ", completion.choices[0].message.content)


# Guided decoding by Regex
def guided_regex_completion(client: OpenAI, model: str):
    prompt = ("What is the capital of France?")

    completion = client.chat.completions.create(
        model=model,
        messages=[{
            "role": "user",
            "content": prompt,
        }],
        extra_body={
            "guided_regex": "(Paris|London)",
        },
    )
    print_completion_details(completion)


class People(BaseModel):
    name: str
    age: int


def guided_json_completion(client: OpenAI, model: str):
    json_schema = People.model_json_schema()

    prompt = ("Generate a JSON with the name and age of one random person.")
    completion = client.chat.completions.create(
        model=model,
        messages=[{
            "role": "user",
            "content": prompt,
        }],
        extra_body={"guided_json": json_schema},
    )
    print_completion_details(completion)


# Guided decoding by JSON using Pydantic schema
class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"


class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: CarType


def guided_car_json_completion(client: OpenAI, model: str):
    json_schema = CarDescription.model_json_schema()

    prompt = ("Generate a JSON with the brand, model and car_type of"
              "the most iconic car from the 90's")
    completion = client.chat.completions.create(
        model=model,
        messages=[{
            "role": "user",
            "content": prompt,
        }],
        extra_body={"guided_json": json_schema},
    )
    print_completion_details(completion)


# Guided decoding by Grammar
def guided_grammar_completion(client: OpenAI, model: str):
    simplified_sql_grammar = """
        root ::= select_statement

        select_statement ::= "SELECT " column " from " table " where " condition

        column ::= "col_1 " | "col_2 "

        table ::= "table_1 " | "table_2 "

        condition ::= column "= " number

        number ::= "1 " | "2 "
    """

    # This may be very slow https://github.com/vllm-project/vllm/issues/12122
    prompt = ("Generate an SQL query to show the 'username' and 'email'"
              "from the 'users' table.")
    completion = client.chat.completions.create(
        model=model,
        messages=[{
            "role": "user",
            "content": prompt,
        }],
        extra_body={"guided_grammar": simplified_sql_grammar},
    )
    print_completion_details(completion)


def main():
    client: OpenAI = OpenAI(
        api_key=openai_api_key,
        base_url=openai_api_base,
    )

    models = client.models.list()
    model: str = models.data[0].id

    print("Guided Regex Completion:")
    guided_regex_completion(client, model)

    print("\nGuided JSON Completion (People):")
    guided_json_completion(client, model)

    print("\nGuided JSON Completion (CarDescription):")
    guided_car_json_completion(client, model)

    print("\nGuided Grammar Completion:")
    guided_grammar_completion(client, model)


if __name__ == "__main__":
    main()

Guided Regex Completion:
reasoning_content:  London
content:  None

Guided JSON Completion (People):
reasoning_content:  {
  "name": "John",
  "age": 30
}
content:  None

Guided JSON Completion (CarDescription):
reasoning_content:  {
  "brand": "Toyota",
  "model": "Supra",
 





















































































































































































































































































































































































































































































































































































































































































































































In [6]:
# SPDX-License-Identifier: Apache-2.0
"""
An example shows how to generate chat completions from reasoning models
like DeepSeekR1.

To run this example, you need to start the vLLM server
with the reasoning parser:

```bash
vllm serve deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B \
    --reasoning-parser deepseek_r1
```

This example demonstrates how to generate chat completions from reasoning models
using the OpenAI Python client library.
"""

from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://192.168.0.166:8000/v1"


def main():
    client = OpenAI(
        api_key=openai_api_key,
        base_url=openai_api_base,
    )

    models = client.models.list()
    model = models.data[0].id

    # Round 1
    messages = [{"role": "user", "content": "9.11 and 9.8, which is greater?"}]
    # ruff: noqa: E501
    # For granite, add: `extra_body={"chat_template_kwargs": {"thinking": True}}`
    response = client.chat.completions.create(model=model, messages=messages)

    reasoning_content = response.choices[0].message.reasoning_content
    content = response.choices[0].message.content

    print("reasoning_content for Round 1:", reasoning_content)
    print("content for Round 1:", content)

    # Round 2
    messages.append({"role": "assistant", "content": content})
    messages.append({
        "role":
        "user",
        "content":
        "How many Rs are there in the word 'strawberry'?",
    })
    response = client.chat.completions.create(model=model, messages=messages)

    reasoning_content = response.choices[0].message.reasoning_content
    content = response.choices[0].message.content

    print("reasoning_content for Round 2:", reasoning_content)
    print("content for Round 2:", content)


if __name__ == "__main__":
    main()

reasoning_content for Round 1: None
content for Round 1: <think>
Okay, so I need to figure out which number is greater between 9.11 and 9.8. Hmm, let's start by recalling how to compare decimal numbers. I remember that when comparing decimals, you start by looking at the whole number part first. If the whole numbers are the same, then you move to the decimal places, comparing each digit from left to right until you find a difference.

Alright, both numbers here are 9. something. So their whole number parts are both 9, which means they're equal up to that point. That means I need to look at the decimal parts to determine which one is larger. 

Let's break them down:

- The first number is 9.11. The decimal part here is 11.
- The second number is 9.8. The decimal part here is 8.

Wait, but hold on. When we compare decimals, isn't the place value important? Like, the first digit after the decimal is tenths, the second is hundredths, right? Wait, actually, no. Wait, 9.11 has two decimal pl